## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [7]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.compat.v1.get_variable(name="W1", shape=(784, 100), initializer=tf.random_normal_initializer(), trainable=True)
        self.W2 = tf.compat.v1.get_variable(name="W2", shape=(100, 10), initializer=tf.random_normal_initializer(), trainable=True)
        self.b1 = tf.compat.v1.get_variable(name="b1", shape=(100, ), trainable=True)
        self.b2 = tf.compat.v1.get_variable(name="b2", shape=(10, ), trainable=True)
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x_prime = tf.reshape(x, [-1, 784])
        h1 = tf.nn.relu(tf.matmul(x_prime, self.W1)+self.b1)
        logits = tf.matmul(h1, self.W2)+self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [8]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [9]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3184364 ; accuracy 0.1193
epoch 1 : loss 2.315961 ; accuracy 0.1203
epoch 2 : loss 2.3134954 ; accuracy 0.121583335
epoch 3 : loss 2.311039 ; accuracy 0.123
epoch 4 : loss 2.3085916 ; accuracy 0.124
epoch 5 : loss 2.306153 ; accuracy 0.1251
epoch 6 : loss 2.3037226 ; accuracy 0.12595
epoch 7 : loss 2.3013 ; accuracy 0.1269
epoch 8 : loss 2.2988849 ; accuracy 0.12793334
epoch 9 : loss 2.2964768 ; accuracy 0.1287
epoch 10 : loss 2.2940757 ; accuracy 0.12945
epoch 11 : loss 2.2916822 ; accuracy 0.13028333
epoch 12 : loss 2.2892947 ; accuracy 0.13105
epoch 13 : loss 2.286914 ; accuracy 0.13213333
epoch 14 : loss 2.284539 ; accuracy 0.1332
epoch 15 : loss 2.2821693 ; accuracy 0.134
epoch 16 : loss 2.2798047 ; accuracy 0.13508333
epoch 17 : loss 2.2774446 ; accuracy 0.13598333
epoch 18 : loss 2.275089 ; accuracy 0.13666667
epoch 19 : loss 2.2727375 ; accuracy 0.13771667
epoch 20 : loss 2.27039 ; accuracy 0.13858333
epoch 21 : loss 2.2680464 ; accuracy 0.13928333
epoch 22 : l